In [167]:
#Importing the required headers.
import pandas as pd
import numpy as np
import csv
import random

In [198]:
#Opening one of the match files.
df = pd.read_csv("./Input/InputMatch3.csv")

#Opening Data files.
BatsmanClusters = pd.read_csv("../Cleaning/OutputBatsmenCleaned.csv", usecols = ['Batsman', 'Cluster'])
BowlerClusters = pd.read_csv("../Cleaning/OutputBowlersCleaned.csv", usecols = ['Bowler', 'Cluster'])
PlayerProbabilites = pd.read_csv("../Cleaning/ProbabilitiesCleaned.csv", usecols = ['Batsman', \
                    'Bowler', 'p(0\'s)', 'p(1\'s)', 'p(2\'s)', 'p(3\'s)', 'p(4\'s)', 'p(6\'s)', 'p(Out)', 'TotalBalls'])
ClusterProbabilities = pd.read_csv("../ClusterToCluster/ClusterProbability.csv", usecols = ['Batsman', \
                    'Bowler', 'p(0\'s)', 'p(1\'s)', 'p(2\'s)', 'p(3\'s)', 'p(4\'s)', 'p(6\'s)', 'p(Out)', 'TotalBalls'])

#To store the match details.
team1_bat_order = list()
team1_bow_order = list()
team2_bat_order = list()
team2_bow_order = list()

#All possible scores.
score_list = [0, 1, 2, 3, 4, 6]

#Collecting the data.
team1_bat_order = df['T1_Bat_Order']
team1_bow_order = df['T1_Bow_Order']
team1_bow_overs = df['T1_Overs']
team2_bat_order = df['T2_Bat_Order']
team2_bow_order = df['T2_Bow_Order']
team2_bow_overs = df['T2_Overs']

In [196]:
#Implementing the required functions.

#Finding the clusters to which the batsman and bowler belong to.
def cluster_number(batsman, bowler):
    batsman_cluster_no = BatsmanClusters[BatsmanClusters.Batsman == batsman]
    if(len(batsman_cluster_no) == 0):
        batsman_cluster_no = 0
    else:
        batsman_cluster_no = batsman_cluster_no.iloc[0]['Cluster']
        
    bowler_cluster_no = BowlerClusters[BowlerClusters.Bowler == bowler]
    if(len(bowler_cluster_no) == 0):
        bowler_cluster_no = 0
    else:
        bowler_cluster_no = bowler_cluster_no.iloc[0]['Cluster']
    
    return batsman_cluster_no, bowler_cluster_no

#Trying to obtain player to player prob. if it is present in the data.
def pvp_plist(batsman, bowler):
    temp = PlayerProbabilites[PlayerProbabilites.Batsman == batsman]
    temp = temp[temp.Bowler == bowler]
    
    if(len(temp)==0):
        return False, None
    else:
        return True, (temp.values[0])[2:9]
    
#Extracting the cluster to cluster probability data.
def cvc_plist(BatsmanClusterNo, BowlerClusterNo):
    temp = ClusterProbabilities[ClusterProbabilities.Batsman == BatsmanClusterNo]
    temp = temp[temp.Bowler == BowlerClusterNo]
    return (temp.values[0])[2:]

#Get a score for a ball based on random number generated. 
def random_pick(some_list, probabilities) :
    np.delete(probabilities, [6])
    #x = random.uniform(0,sum(probabilities))
    x = np.random.normal(0.55, 0.25, 1)[0]
    cumulative_probability = 0.0
    for item, item_probability in zip(some_list, probabilities):
        cumulative_probability += item_probability
        if x < cumulative_probability: break
    return item

#The simulator function.
def innings(bat_order, bow_order, overs,inn):
    tot_wickets = 0
    striker = 1
    non_striker = 0
    striker_notout = 1
    non_striker_notout = 1
    
    #Which bowler is bowling in which over.
    #bow_index_order = [random.randint(0,5) for x in range(0,20)]
    bow_index_order = overs
    x = bow_index_order[0]

    total_runs = 0
    k = -1#Over number
    
    #For all possible balls in an innings.
    for i in range(0,120) : 
        # Swap batsmen and change bowlers after every over.
        if(i%6 == 0):
            k += 1
            x = bow_index_order[k]
            #Swap the Striker and Non Striker data.
            tmp_striker = striker
            tmp_notout = striker_notout
            striker = non_striker
            striker_notout = non_striker_notout
            non_striker = tmp_striker
            non_striker_notout = tmp_notout

        #Update data.
        curr_bat = bat_order[striker].rstrip() #Striker
        other_bat = bat_order[non_striker].rstrip()#Non Striker
        curr_bow = bow_order[x].rstrip() #Bowler
        
        #Prediction for the current ball.
        exists, pvp_p_list = pvp_plist(curr_bat, curr_bow)
        if exists :
            striker_notout *= float(1 - (pvp_p_list[6]))
            prediction = random_pick(score_list, pvp_p_list)
            
        else :
            bat_c_num, bow_c_num = cluster_number(curr_bat, curr_bow)
            cvc_p_list = cvc_plist(bat_c_num, bow_c_num)
            striker_notout *= float(1 - (cvc_p_list[6]))
            prediction = random_pick(score_list, cvc_p_list)
            
        print("Over: ", str(i//6)+'.'+str(i%6),"| Bowler: ", curr_bow)
        print("Striker: ", curr_bat, "| Non-Striker: ", other_bat)
        
        #Checking if the wicket is probable to fall.
        if(striker_notout < 0.5):
            tot_wickets += 1
            striker = max(striker,non_striker) + 1
            striker_notout = 1
            print("Result: Wicket!\n")
            if(striker > 10):
                if(tot_wickets >= 10):
                    break

        elif((prediction == 0) or (prediction == 2) or (prediction == 4) or (prediction == 6)):
            total_runs += prediction
            print("Result:",prediction, "run.\n")

        elif((prediction == 1) or (prediction == 3)):
            total_runs += prediction
            #Swapping the Striker and Non Striker data.
            tmp_striker = striker
            tmp_notout = striker_notout
            striker = non_striker
            striker_notout = non_striker_notout
            non_striker = tmp_striker
            non_striker_notout = tmp_notout
            print("Result:", prediction, "run.\n")
        
        #If it is first innings then save the score.
        if(inn == 1):
            global first_inn_score
            first_inn_score = total_runs

        #If it is second innings and if the team has chased down the target then stop.
        if((inn == 2) and (total_runs > first_inn_score)):
            break

    #Final Result.
    num_of_overs_played = str(int((i+1)/6)) + "." + str((i+1)%6)
    return total_runs, tot_wickets, str(total_runs)+"/"+str(tot_wickets)+"\nOvers : "+ num_of_overs_played

In [197]:
#Simulation Driver Function.
print("IPL MATCH SIMULATION")
print("-------------------------------------------------------------------------------------")

print("\nINNINGS 1:\n")
first_innings_score, wickets1, formatted_score1 = innings(team1_bat_order, team2_bow_order, team2_bow_overs,1)
print ("Team 1 Score :\n" + formatted_score1)

print("-------------------------------------------------------------------------------------")
print("\nINNINGS 2:\n")
second_innings_score, wickets2, formatted_score2 = innings(team2_bat_order, team1_bow_order, team1_bow_overs,2)
print ("Team 2 Score :\n" + formatted_score2)

print("\n-------------------------------------------------------------------------------------")
print("MATCH RESULT:")
if first_innings_score > second_innings_score :
    print ("\nTeam 1 wins!")
    print ("By "+str(first_innings_score - second_innings_score)+" runs")
elif second_innings_score > first_innings_score :
    print ("\nTeam 2 wins!")
    print ("By "+ str(10 - wickets2) +" wickets")
else :
    print ("\nMatch Tied.")

IPL MATCH SIMULATION
-------------------------------------------------------------------------------------

INNINGS 1:

Over:  0.0 | Bowler:  V Kumar
Striker:  B McCullum | Non-Striker:  Q de Kock
Result: 1 run.

Over:  0.1 | Bowler:  V Kumar
Striker:  Q de Kock | Non-Striker:  B McCullum
Result: 0 run.

Over:  0.2 | Bowler:  V Kumar
Striker:  Q de Kock | Non-Striker:  B McCullum
Result: 1 run.

Over:  0.3 | Bowler:  V Kumar
Striker:  B McCullum | Non-Striker:  Q de Kock
Result: 1 run.

Over:  0.4 | Bowler:  V Kumar
Striker:  Q de Kock | Non-Striker:  B McCullum
Result: 6 run.

Over:  0.5 | Bowler:  V Kumar
Striker:  Q de Kock | Non-Striker:  B McCullum
Result: 4 run.

Over:  1.0 | Bowler:  P Chawla
Striker:  B McCullum | Non-Striker:  Q de Kock
Result: 0 run.

Over:  1.1 | Bowler:  P Chawla
Striker:  B McCullum | Non-Striker:  Q de Kock
Result: 1 run.

Over:  1.2 | Bowler:  P Chawla
Striker:  Q de Kock | Non-Striker:  B McCullum
Result: 0 run.

Over:  1.3 | Bowler:  P Chawla
Striker: 

Over:  15.5 | Bowler:  A Russell
Striker:  Mandeep Singh | Non-Striker:  U Yadav
Result: 1 run.

Over:  16.0 | Bowler:  M Johnson
Striker:  Mandeep Singh | Non-Striker:  U Yadav
Result: 0 run.

Over:  16.1 | Bowler:  M Johnson
Striker:  Mandeep Singh | Non-Striker:  U Yadav
Result: 1 run.

Over:  16.2 | Bowler:  M Johnson
Striker:  U Yadav | Non-Striker:  Mandeep Singh
Result: 1 run.

Over:  16.3 | Bowler:  M Johnson
Striker:  Mandeep Singh | Non-Striker:  U Yadav
Result: 0 run.

Over:  16.4 | Bowler:  M Johnson
Striker:  Mandeep Singh | Non-Striker:  U Yadav
Result: 1 run.

Over:  16.5 | Bowler:  M Johnson
Striker:  U Yadav | Non-Striker:  Mandeep Singh
Result: 4 run.

Over:  17.0 | Bowler:  S Narine
Striker:  Mandeep Singh | Non-Striker:  U Yadav
Result: Wicket!

Over:  17.1 | Bowler:  S Narine
Striker:  Y Chahal | Non-Striker:  U Yadav
Result: 1 run.

Over:  17.2 | Bowler:  S Narine
Striker:  U Yadav | Non-Striker:  Y Chahal
Result: 4 run.

Over:  17.3 | Bowler:  S Narine
Striker:  

Result: 1 run.

Over:  11.1 | Bowler:  Y Chahal
Striker:  A Russell | Non-Striker:  D Karthik
Result: 1 run.

Over:  11.2 | Bowler:  Y Chahal
Striker:  D Karthik | Non-Striker:  A Russell
Result: 1 run.

Over:  11.3 | Bowler:  Y Chahal
Striker:  A Russell | Non-Striker:  D Karthik
Result: 2 run.

Over:  11.4 | Bowler:  Y Chahal
Striker:  A Russell | Non-Striker:  D Karthik
Result: 0 run.

Over:  11.5 | Bowler:  Y Chahal
Striker:  A Russell | Non-Striker:  D Karthik
Result: 2 run.

Over:  12.0 | Bowler:  Kulwant Khejroliya
Striker:  D Karthik | Non-Striker:  A Russell
Result: 1 run.

Over:  12.1 | Bowler:  Kulwant Khejroliya
Striker:  A Russell | Non-Striker:  D Karthik
Result: 1 run.

Over:  12.2 | Bowler:  Kulwant Khejroliya
Striker:  D Karthik | Non-Striker:  A Russell
Result: 0 run.

Over:  12.3 | Bowler:  Kulwant Khejroliya
Striker:  D Karthik | Non-Striker:  A Russell
Result: Wicket!

Over:  12.4 | Bowler:  Kulwant Khejroliya
Striker:  V Kumar | Non-Striker:  A Russell
Result: 1 r